<a href="https://colab.research.google.com/github/marco-siino/text_preprocessing_impact/blob/main/20N_DS/SVM_20N_TextPreProImpact_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text preprocessing worth the time: A comparative survey on the impact of common techniques on NLP model performances.
- - -
SVM ON 20NewsGroup DS EXPERIMENTS NOTEBOOK
- - -
Support Vector Machine on 20Newsgroup Dataset.
Code by M. Siino.

From the paper: "Text preprocessing worth the time: A comparative survey on the impact of common techniques on NLP model performances." by M.Siino et al.



## Importing modules.

In [1]:
import matplotlib.pyplot as plt
import os
import random
import re
import shutil
import string
import tensorflow as tf
import nltk

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from google.colab import files
from io import open
from numpy.random import seed
import numpy as np
from pathlib import Path
from sklearn import svm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from textblob import TextBlob
nltk.download('stopwords')
nltk.download('punkt')

os.environ['TF_CUDNN_DETERMINISTIC']='true'
os.environ['TF_DETERMINISTIC_OPS']='true'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Importing DS and extract in current working directory.

In [2]:
urlDataset = "https://github.com/marco-siino/text_preprocessing_impact/raw/main/20N_DS/20news-bydate.tar.gz"

dataset = tf.keras.utils.get_file("20news-bydate.tar.gz", urlDataset,
                                    extract=True, archive_format='tar',cache_dir='.',
                                    cache_subdir='')

test_dir = '20news-bydate-test'
train_dir = '20news-bydate-train'

14464277/14464277 [==============================] - 0s 0us/step


## Build folders hierarchy to use Keras folders preprocessing function.

In [3]:
def walk_dir(targetdir,topdown=True):
    for root, dirs, files in os.walk(targetdir, topdown):
        for subfolder in dirs:
          #print(subfolder)
          #print(root)
          #print(dirs)
          #print(files)
          for subroot, subdirs, files in os.walk(root+'/'+subfolder, topdown):
            #print(files)
            for name in files:
              #print(name)
              os.rename(root+'/'+subfolder+'/'+name, root+'/'+subfolder+'/'+name+'.txt')

walk_dir(test_dir)
walk_dir(train_dir)

## Generate full training set.



In [4]:
# Generate full randomized training set.
batch_size=1

train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    shuffle=False
    )

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size,
    shuffle=False
    )

train_ds=train_ds.shuffle(11314,seed=1, reshuffle_each_iteration=False)
test_ds=test_ds.shuffle(7532,seed=1, reshuffle_each_iteration=False)

train_ds_size=len(train_ds)
test_ds_size=len(test_ds)

train_ds = train_ds.take(200)
test_ds = test_ds.take(50)


#train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=20)))
#test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, depth=20)))

Found 11314 files belonging to 20 classes.
Found 7532 files belonging to 20 classes.


## Functions to pre-process source text. (A detailed discussion on our paper)

In [5]:
# Do-Nothing preprocessing function.
def DON(input_data):
  tag_open_CDATA_removed = tf.strings.regex_replace(input_data, '<\!\[CDATA\[', ' ')
  tag_closed_CDATA_removed = tf.strings.regex_replace(tag_open_CDATA_removed,'\]{1,}>', ' ')
  tag_author_lang_en_removed = tf.strings.regex_replace(tag_closed_CDATA_removed,'<author lang="en">', ' ')
  tag_closed_author_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'</author>', ' ')
  tag_open_documents_removed = tf.strings.regex_replace(tag_closed_author_removed,'<documents>\n(\t){0,2}', '')
  output_data = tf.strings.regex_replace(tag_open_documents_removed,'</documents>\n(\t){0,2}', ' ')
  return output_data

# Lowercasing preprocessing function.
def LOW(input_data):
  return tf.strings.lower(DON(input_data))

# Removing Stop Words function.
def RSW(input_data):
  output_data = DON(input_data)

  #print("\n\nInput data è il seguente tensore:")
  #print(output_data)

  #print("Lo converto in stringa e diventa:")
  # Il seguente try per l'adattamento del ts. Nell'except caso della simulazione vera e propria.
  try:
    input_string=output_data[0]

  # # # # # # # Questo è il caso della chiamata a funzione per la simulazione vera e propria.
  except:
    #print("\n\n****CASO DELLA SIMULAZIONE VERA E PROPRIA****\n\n")
    #print("\nQuesto è il contenuto di output data in caso di simulazione")
    #print(output_data)
    input_string=output_data

    try:
      input_string = input_string.numpy()

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      #print("\nEstraendo il contenuto del tensore risulta:")
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [word for word in blob if word not in stopwords.words('english')]
    #print("tolte le stopword inglesi diventa:")

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)

    output_tensor=tf.constant(output_string)
    #print(output_tensor)

    return output_tensor

   # # # # # # # Questo è il caso dell'adattamento del TS.
  else:

    try:

      # input_string = input_string.numpy() [0]
      input_string = input_string.numpy()

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [word for word in blob if word not in stopwords.words('english')]
    #print("Tolte le stopword inglesi diventa:")

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)

    output_tensor=tf.constant([[output_string]])
    #print(output_tensor)

    return output_tensor

  return output_data

# Porter Stemmer preprocessing function.
def STM(input_data):
  output_data = DON(input_data)
  stemmer = PorterStemmer()

  #print("\n\nInput data è il seguente tensore:")
  #print(output_data)

  #print("Lo converto in stringa e diventa:")
  # Il seguente try per l'adattamento del ts. Nell'except caso della simulazione vera e propria.
  try:
    input_string=output_data[0]

  # # # # # # # Questo è il caso della chiamata a funzione per la simulazione vera e propria.
  except:
    #print("\n\n****CASO DELLA SIMULAZIONE VERA E PROPRIA****\n\n")
    #print("\nQuesto è il contenuto di output data in caso di simulazione")
    #print(output_data)
    input_string=output_data

    try:
      input_string = input_string.numpy()

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      #print("\nEstraendo il contenuto del tensore risulta:")
      #print(input_string)
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [stemmer.stem(word) for word in blob]

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)

    output_tensor=tf.constant(output_string)
    #print(output_tensor)

    return output_tensor

   # # # # # # # Questo è il caso dell'adattamento del TS.
  else:

    try:
      #input_string = input_string.numpy()[0]
      input_string = input_string.numpy()
      #print(input_string)

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [stemmer.stem(word) for word in blob]

    output_string = (' '.join(word for word in outputlist))

    output_tensor=tf.constant([[output_string]])
    #print(output_tensor)

    return output_tensor

  return output_data

## Define the combined preprocessing functions. (The base functions are: DON, LOW, RSW and STM).

In [6]:
## SECTION WITH PAIRS OF PREPRO FUNCTIONS. APPLICATION ORDER MATTERS (...IN FOLLOWING SECTIONS TOO).
#...5
def LOW_RSW(input_data):
  return RSW(LOW(input_data))

# 6
def LOW_STM(input_data):
  return STM(LOW(input_data))

# 7
def RSW_LOW(input_data):
  return LOW(RSW(input_data))

# 8
def RSW_STM(input_data):
  return STM(RSW(input_data))

# 9
def STM_LOW(input_data):
  return LOW(STM(input_data))

# 10
def STM_RSW(input_data):
  return RSW(STM(input_data))

# 11
def LOW_STM_RSW(input_data):
  return RSW(STM(LOW(input_data)))

# 12
def LOW_RSW_STM(input_data):
  return STM(RSW(LOW(input_data)))

# 13
def STM_LOW_RSW(input_data):
  return RSW(LOW(STM(input_data)))

# 14
def STM_RSW_LOW(input_data):
  return LOW(RSW(STM(input_data)))

# 15
def RSW_LOW_STM(input_data):
  return STM(LOW(RSW(input_data)))

# 16
def RSW_STM_LOW(input_data):
  return LOW(STM(RSW(input_data)))

## Get the length of the longest sample in training set. Then adapt text.



In [7]:
max_features = 0
def preprocess_and_adapt_ts(preprocessing_function,training_set):
  # Set a large sequence length to find the longest sample in the training set.
  sequence_length = 15000
  vectorize_layer = TextVectorization(
      standardize=preprocessing_function,
      output_mode='int',
      output_sequence_length=sequence_length,
      encoding='ISO-8859-1')

  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  #vectorize_layer.get_vocabulary()

  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
  model.add(vectorize_layer)

  longest_sample_length=1

  for element in training_set:
    authorDocument=element[0]
    label=element[1]

    #print("Sample considered is: ", authorDocument[0].numpy())
    #print("Preprocessed: ", str(custom_standardization(authorDocument[0].numpy())))
    #print("And has label: ", label[0].numpy())

    out=model(authorDocument)
    # Convert token list to numpy array.
    token_list = out.numpy()[0]
    token_list = np.trim_zeros(token_list,'b')
    if longest_sample_length < len(token_list):
      longest_sample_length = len(token_list)

  print("Length of the longest sample is:", longest_sample_length)

  # After tokenization longest_sample_length covers all the document lenghts in our dataset.
  sequence_length = longest_sample_length

  vectorize_layer = TextVectorization(
      standardize=preprocessing_function,
      output_mode='int',
      output_sequence_length=sequence_length,
      encoding='ISO-8859-1')

  # Finally adapt the vectorize layer.
  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  global max_features
  max_features=len(vectorize_layer.get_vocabulary()) + 1
  return vectorize_layer

## Define a dictionary with -> function_names:prepro_function_caller. And a dictionary to store model results.




In [8]:
model_results = {}
prepro_functions_dict_base = {
    'DON':DON,
    'LOW':LOW,
    'RSW':RSW,
    'STM':STM
    }

# 3 prepro functions = 15 combs...+1 for do_nothing

prepro_functions_dict_comb = {
    # 1. Do nothing
    'DON': DON,
    # 2. Lowercasing
    'LOW':LOW,
    # 3. Removing Stopwords
    'RSW':RSW,
    # 4. Porter Stemming
    'STM':STM,
    # 5. LOW->RSW
    'LOW_RSW':LOW_RSW,
    # 6. LOW->STM
    'LOW_STM':LOW_STM,
    # 7. RSW->LOW
    'RSW_LOW':RSW_LOW,
    # 8. RSW->STM
    'RSW_STM':RSW_STM,
    # 9. STM->LOW
    'STM_LOW':STM_LOW,
    # 10. STM->RSW
    'STM_RSW':STM_RSW,
    # 11. LOW->STM->RSW
    'LOW_STM_RSW':LOW_STM_RSW,
    # 12. LOW->RSW->STM
    'LOW_RSW_STM':LOW_RSW_STM,
    # 13. STM->LOW->RSW
    'STM_LOW_RSW':STM_LOW_RSW,
    # 14. STM->RSW->LOW
    'STM_RSW_LOW':STM_RSW_LOW,
    # 15. RSW->LOW->STM
    'RSW_LOW_STM':RSW_LOW_STM,
    # 16. RSW->STM->LOW
    'RSW_STM_LOW':RSW_STM_LOW
}

for key in prepro_functions_dict_comb:
  print(key)
  model_results[key]=[]

DON
LOW
RSW
STM
LOW_RSW
LOW_STM
RSW_LOW
RSW_STM
STM_LOW
STM_RSW
LOW_STM_RSW
LOW_RSW_STM
STM_LOW_RSW
STM_RSW_LOW
RSW_LOW_STM
RSW_STM_LOW


## Models definition and evaluation.




In [9]:
for key in prepro_functions_dict_comb:
    print("\n\n* * * * EVALUATION USING", key, "AS PREPROCESSING FUNCTION * * * *")

    # Preprocess training set to build a dictionary.
    vectorize_layer = preprocess_and_adapt_ts(prepro_functions_dict_comb[key],train_ds)

    print("\n\n***** FINITO DI PROCESSARE E ADATTARE IL TRAINING SET, INIZIA LA SIMULAZIONE *******")
    # Print a raw and a preprocessed sample.
    for element in train_ds:
      authorDocument=element[0]
      label=element[1]

      print("Sample considered is: ", authorDocument[0])
      print("Preprocessed: ", str(prepro_functions_dict_comb[key](authorDocument[0].numpy())))
      break

    # # # - - - - - MODELS DEFINITION AND EVALUATION - - - - - # # #

    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
    model.add(vectorize_layer)

    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    # --- SVM SECTION START --- #

    training_labels=[]
    training_samples=[]

    max_features=len(vectorize_layer.get_vocabulary()) + 1

    for element in train_ds:
      authorDocument=element[0]
      label=element[1]

      #print("Sample considered is: ", authorDocument[0])
      #print("Preprocessed: ", str(custom_standardization(authorDocument[0].numpy())))
      #print("And has label: ", label[0].numpy())

      text_vect_layer_model = tf.keras.Model(inputs=model.input,
                                          outputs=model.layers[0].output)
      text_vect_out = text_vect_layer_model(authorDocument)

      training_labels.append(label[0].numpy())
      current_sample=np.zeros(max_features)
      for current_token in text_vect_out[0][:].numpy():
        #print(current_token,end=' ')
        #print(vectorize_layer.get_vocabulary()[current_token])
        current_sample[current_token]+=1
      training_samples.append(current_sample)
      #break

    training_labels=np.array(training_labels)
    training_samples=np.array(training_samples)
    #print("\nLE LABELS DEI CAMPIONI DI TRAINING SONO:")
    #print(training_labels)
    #print("\nI SAMPLE DI TRAINING DOPO LA TEXT VECTORIZATION SONO:")
    #print(training_samples)

    test_labels=[]
    test_samples=[]

    for element in test_ds:
      authorDocument=element[0]
      label=element[1]

      text_vect_layer_model = tf.keras.Model(inputs=model.input,
                                          outputs=model.layers[0].output)
      text_vect_out = text_vect_layer_model(authorDocument)

      test_labels.append(label[0].numpy())
      current_sample=np.zeros(max_features)
      for current_token in text_vect_out[0][:].numpy():
        current_sample[current_token]+=1
      test_samples.append(current_sample)

    test_labels=np.array(test_labels)
    test_samples=np.array(test_samples)

    SVM = svm.SVC(C=0.5, kernel='linear', gamma='auto')
    SVM.fit(training_samples,training_labels)
    # predict the labels on training set
    #predictions_SVM = SVM.predict(training_samples)
    # Use accuracy_score function to get the accuracy
    #model_results[key]['SVM_train']=SVM.score(training_samples,training_labels)
    #print("SVM Accuracy Score on Training set -> ",model_results[key]['SVM_train'])

    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(test_samples)
    # Use accuracy_score function to get the accuracy
    model_results[key].append(SVM.score(test_samples,test_labels))
    print("SVM Accuracy Score on Test set -> ",model_results[key])

    # --- SVM SECTION END --- #

    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    # # # - - - - - MODEL DEFINITION AND EVALUATION END  - - - - - # # #



* * * * EVALUATION USING DON AS PREPROCESSING FUNCTION * * * *
Length of the longest sample is: 7969


***** FINITO DI PROCESSARE E ADATTARE IL TRAINING SET, INIZIA LA SIMULAZIONE *******
Sample considered is:  tf.Tensor(b"From: wrat@unisql.UUCP (wharfie)\nSubject: Re: Too fast\nOrganization: UniSQL, Inc., Austin, Texas, USA\nLines: 9\n\nIn article <1993Apr14.152328.15997@magnus.acs.ohio-state.edu> jnielsen@magnus.acs.ohio-state.edu (John F Nielsen) writes:\n>There may be a case where a speed limit sign is not necessary. But take\n>them away entirely?\n\n\tYeah, you're right.  Doing away with speed limits would just\nmean huge tax increases as municipalities tried to make up for the\nrevenue they used to gouge from passing motorists.\n\n\n", shape=(), dtype=string)
Preprocessed:  tf.Tensor(b"From: wrat@unisql.UUCP (wharfie)\nSubject: Re: Too fast\nOrganization: UniSQL, Inc., Austin, Texas, USA\nLines: 9\n\nIn article <1993Apr14.152328.15997@magnus.acs.ohio-state.edu> jnielsen@magnus.

## Now show compact results in a table.

In [10]:
print(" PREPRO FUNCTION    |  Test Accuracy   |",end = '')

print("\n")
for prepro_func in prepro_functions_dict_comb:
  #print(prepro_func,"\t\t\t",format(round(model_results[prepro_func][0],4),'.4f'),"\t\t",end='')
  result = format(round(model_results[prepro_func][0],4),'.4f')
  print(f'{prepro_func:27}{ result :12}')
  print("\n")


 PREPRO FUNCTION    |  Test Accuracy   |

DON                        0.1600      


LOW                        0.1400      


RSW                        0.1000      


STM                        0.1600      


LOW_RSW                    0.1400      


LOW_STM                    0.1200      


RSW_LOW                    0.1200      


RSW_STM                    0.1600      


STM_LOW                    0.1200      


STM_RSW                    0.1400      


LOW_STM_RSW                0.1400      


LOW_RSW_STM                0.1400      


STM_LOW_RSW                0.1400      


STM_RSW_LOW                0.1400      


RSW_LOW_STM                0.1400      


RSW_STM_LOW                0.1600      


